In [13]:
import pandas as pd


# Определение ставок VAT и duty
vat_rates = {
    'DE': 0.19,            # Germany
    'PT': 0.23,            # Portugal (mainland, Madeira, Azores)
    'ES': 0.21,            # Spain (except Canary Islands)
    'IE': 0.23,            # Ireland
    'SE': 0.25,            # Sweden
    'NL': 0.21,            # Netherlands
    'DK': 0.25,            # Denmark (mainland only)
    'FI': 0.255,           # Finland (except Åland Islands, raised in 2024)
    'IT': 0.22,            # Italy (except Livigno, Campione d’Italia, Lake Lugano waters)
    'AT': 0.20,            # Austria
    'BE': 0.21,            # Belgium
    'EE': 0.22             # Estonia (until July); use 0.24 for returns post-July 2025
}




In [14]:
df = pd.read_csv("TED BAKER DUTY CLAIM BACK Jul-Sep v2.csv")

df = df[~df['Consignee Country'].isin(['IC', 'CH'])] # exclude all the non-eu countries

# convert placeholder strings to real missing values, then fill from Parcel ID
df['MRN'] = df['MRN'].replace(['#N/A', 'N/A', 'NA', 'na', ''], pd.NA)
df.loc[df['MRN'].isna(), 'MRN'] = df.loc[df['MRN'].isna(), 'Parcel ID']


# Ensure correct consignment grouping
df['Consignment Value'] = df.groupby('MRN')['Line Item Unit Price'].transform('sum')

#df = df[df['Consignment Value'] <= 150]



In [15]:
# Calculate net quantity and price with VAT per row
df['Net Quantity'] = df['Line Item Quantity Imported'] - df['Line Item Quantity Returned']
df['VAT Rate'] = df['Consignee Country'].map(vat_rates)
df['Total Paid'] = df['Net Quantity'] * df['Line Item Unit Price']
df['VAT Paid'] = df['Net Quantity'] * df['Line Item Unit Price'] * df['VAT Rate']


# After mapping VAT rates
if df['VAT Rate'].isna().any():
    print("WARNING: Some countries missing VAT rates!")
    print(df[df['VAT Rate'].isna()]['Consignee Country'].unique())


# Group by country and sum the prices
result_df = df.groupby(['Consignee Country', 'VAT Rate']).agg({'VAT Paid': 'sum', 'Total Paid': 'sum'}).reset_index()
result_df.columns = ['Country', 'VAT Rate', 'Total VAT Paid', 'Total Items Price']

print(result_df)

print(f"\nTotal VAT Paid: {result_df['Total VAT Paid'].sum()}")

result_df.to_excel('lv_declaration_data.xlsx', index=False)


   Country  VAT Rate  Total VAT Paid  Total Items Price
0       AT     0.200       1156.6000            5783.00
1       BE     0.210        496.0200            2362.00
2       DE     0.190      29796.5600          156824.00
3       DK     0.250       3074.7500           12299.00
4       ES     0.210       8916.6000           42460.00
5       FI     0.255       7499.0400           29408.00
6       IE     0.230      70696.2500          307375.00
7       IT     0.220       3114.3200           14156.00
8       NL     0.210      19640.8695           93527.95
9       PT     0.230       4148.2800           18036.00
10      SE     0.250       1258.0000            5032.00

Total VAT Paid: 149797.2895
